In [ ]:
#
# Empaquetado del entrenamiento del modelo
#
def train_estimator(alpha=0.5, l1_ratio=0.5, verbose=1):
    """
    # Función completa para entrenar un modelo ElasticNet
    # 
    # Parámetros:
    # - alpha: Parámetro de regularización que controla la fuerza de la penalización (default=0.5)
    # - l1_ratio: Proporción de penalización L1 frente a L2 (default=0.5)
    # - verbose: Si es > 0, imprime métricas de evaluación (default=1)
    #
    # Esta función:
    # 1. Carga datos de vinos desde URL
    # 2. Divide los datos en conjuntos de entrenamiento y prueba
    # 3. Entrena un modelo ElasticNet con los parámetros proporcionados
    # 4. Evalúa el modelo con métricas MSE, MAE y R2
    # 5. Guarda el modelo si es mejor que el modelo anteriormente guardado
    """

    import os
    import pickle

    import pandas as pd
    from sklearn.linear_model import ElasticNet
    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
    from sklearn.model_selection import train_test_split

    # Carga de datos desde URL - Dataset de calidad de vinos
    url = "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
    df = pd.read_csv(url, sep=";")

    # Separación de características (X) y variable objetivo (y)
    y = df["quality"]
    x = df.copy()
    x.pop("quality")

    # División de datos en conjuntos de entrenamiento (75%) y prueba (25%)
    (x_train, x_test, y_train, y_test) = train_test_split(
        x,
        y,
        test_size=0.25,
        random_state=0,
    )

    # Creación y entrenamiento del modelo ElasticNet
    estimator = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=12345)

    estimator.fit(x_train, y_train)
    y_pred = estimator.predict(x_test)

    # Cálculo de métricas de evaluación
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Mostrar métricas si verbose > 0
    if verbose > 0:
        print(estimator, ":", sep="")
        print(f"  MSE: {mse}")
        print(f"  MAE: {mae}")
        print(f"  R2: {r2}")

    # Comprobar si existe un modelo guardado previamente
    if not os.path.exists("estimator.pickle"):
        saved_estimator = None
    else:
        with open("estimator.pickle", "rb") as file:
            saved_estimator = pickle.load(file)

    # Guardar el modelo actual si es mejor que el anterior o si no existe modelo previo
    if saved_estimator is None or estimator.score(
        x_test, y_test
    ) > saved_estimator.score(x_test, y_test):
        with open("estimator.pickle", "wb") as file:
            pickle.dump(estimator, file)


In [ ]:
#
# Experimento
#
# Prueba del modelo con parámetros específicos:
# - alpha = 0.2: Intensidad de regularización moderada-baja
# - l1_ratio = 0.2: Mayor peso a la regularización L2 que a la L1
train_estimator(0.2, 0.2)




ElasticNet(alpha=0.2, l1_ratio=0.2, random_state=12345):
  MSE: 0.43869119518947153
  MAE: 0.5236106762028768
  R2: 0.2822387414965034


In [ ]:
#
# Experimento
#
# Prueba del modelo con parámetros diferentes:
# - alpha = 0.5: Intensidad de regularización media
# - l1_ratio = 0.5: Balance equitativo entre regularización L1 y L2 (ElasticNet balanceado)
train_estimator(0.5, 0.5)

ElasticNet(alpha=0.5, random_state=12345):
  MSE: 0.5294843132862007
  MAE: 0.5894666734018875
  R2: 0.13368827268570616


In [ ]:
#
# Uso del modelo en productivo
#
def use_estimator():
    """
    # Función para utilizar el modelo entrenado en un entorno productivo
    #
    # Esta función:
    # 1. Carga datos locales desde un archivo CSV
    # 2. Prepara las características (elimina la columna objetivo)
    # 3. Carga el modelo guardado anteriormente
    # 4. Realiza predicciones con el modelo cargado
    # 5. Retorna las predicciones
    """

    import pandas as pd
    import pickle

    # Carga de datos desde archivo local en lugar de URL
    # url = "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
    df = pd.read_csv('data.csv', sep=";")

    # Separación de características y variable objetivo
    y = df["quality"]
    x = df.copy()
    x.pop("quality")

    # Carga del modelo guardado previamente
    with open("estimator.pickle", "rb") as file:
        estimator = pickle.load(file)

    # Realización de predicciones con el modelo
    y_pred = estimator.predict(x)

    return y_pred

# Ejecución de la función y obtención de predicciones
use_estimator()





array([5.04471258, 5.09836647, 5.20264959, ..., 5.94512405, 5.44101164,
       6.07251942], shape=(1599,))

In [ ]:
#
# Carga de datos
#
def load_data():
    """
    # Función para cargar los datos del dataset de vinos
    #
    # Esta función:
    # 1. Carga datos desde un archivo CSV local
    # 2. Separa los datos en características (X) y variable objetivo (y)
    # 3. Retorna las características y la variable objetivo como objetos separados
    #
    # Returns:
    #   x: DataFrame con las características del vino (como acidez, pH, alcohol, etc.)
    #   y: Serie con la calidad del vino (variable objetivo a predecir)
    """

    import pandas as pd

    # Carga de datos desde archivo local en lugar de URL
    # url = "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
    df = pd.read_csv('data.csv', sep=";")

    # Separación de variables objetivo y características
    y = df["quality"]
    x = df.copy()
    x.pop("quality")

    return x, y



In [ ]:
# Particionamiento de datos
#
def make_train_test_split(x, y):
    """
    # Función para dividir los datos en conjuntos de entrenamiento y prueba
    #
    # Parámetros:
    #   x: DataFrame con las características del conjunto de datos
    #   y: Serie con la variable objetivo
    #
    # Esta función:
    # 1. Divide los datos en 75% para entrenamiento y 25% para prueba
    # 2. Utiliza una semilla aleatoria fija (random_state=0) para garantizar reproducibilidad
    #
    # Returns:
    #   x_train: Características para entrenamiento
    #   x_test: Características para prueba
    #   y_train: Variable objetivo para entrenamiento
    #   y_test: Variable objetivo para prueba
    """

    from sklearn.model_selection import train_test_split

    (x_train, x_test, y_train, y_test) = train_test_split(
        x,
        y,
        test_size=0.25,  # 25% de los datos para prueba
        random_state=0,  # Semilla aleatoria fija para reproducibilidad
    )
    return x_train, x_test, y_train, y_test




In [ ]:
# Cálculo de metricas de evaluación
#
def eval_metrics(y_true, y_pred):
    """
    # Función para calcular métricas de evaluación del modelo
    #
    # Parámetros:
    #   y_true: Valores reales de la variable objetivo
    #   y_pred: Valores predichos por el modelo
    #
    # Esta función calcula tres métricas importantes:
    # 1. MSE (Mean Squared Error): Error cuadrático medio - mide el promedio de los errores al cuadrado
    # 2. MAE (Mean Absolute Error): Error absoluto medio - mide el promedio de los errores absolutos
    # 3. R2 (R cuadrado): Coeficiente de determinación - mide la proporción de varianza explicada (1 es perfecto)
    #
    # Returns:
    #   mse, mae, r2: Tupla con las tres métricas calculadas
    """

    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    return mse, mae, r2




In [ ]:
#
# Reporte de métricas de evaluación
#
def report(estimator, mse, mae, r2):
    """
    # Función para imprimir un reporte de las métricas de evaluación del modelo
    #
    # Parámetros:
    #   estimator: Modelo entrenado (para mostrar sus características)
    #   mse: Error cuadrático medio
    #   mae: Error absoluto medio
    #   r2: Coeficiente de determinación R²
    #
    # Esta función imprime:
    # 1. Información sobre el estimador utilizado
    # 2. Las tres métricas principales de evaluación (MSE, MAE, R2)
    # 
    # Valores R² más cercanos a 1 indican mejor ajuste del modelo
    """

    print(estimator, ":", sep="")
    print(f"  MSE: {mse}")
    print(f"  MAE: {mae}")
    print(f"  R2: {r2}")


In [ ]:
#
# Almacenamiento del modelo
#
def save_best_estimator(estimator):
    """
    # Función para guardar el mejor modelo en disco
    #
    # Parámetros:
    #   estimator: El modelo entrenado que se desea guardar
    #
    # Esta función:
    # 1. Serializa el modelo utilizando la biblioteca pickle
    # 2. Guarda el modelo serializado en un archivo llamado "estimator.pickle"
    # 3. Este archivo puede ser cargado posteriormente para hacer predicciones
    #
    # El guardado del modelo permite usarlo más tarde sin necesidad de reentrenarlo
    """

    import os
    import pickle

    with open("estimator.pickle", "wb") as file:
        pickle.dump(estimator, file)



In [ ]:
# Carga del modelo
#
def load_best_estimator():
    """
    # Función para cargar el mejor modelo guardado previamente
    #
    # Esta función:
    # 1. Verifica si existe el archivo "estimator.pickle" que contiene el modelo
    # 2. Si existe, deserializa el modelo usando pickle y lo retorna
    # 3. Si no existe, retorna None
    #
    # Returns:
    #   estimator: El modelo cargado, o None si no existe un modelo guardado
    #
    # Esto permite reutilizar modelos entrenados anteriormente sin necesidad de reentrenarlos
    """

    import os
    import pickle

    if not os.path.exists("estimator.pickle"):
        return None
    with open("estimator.pickle", "rb") as file:
        estimator = pickle.load(file)

    return estimator



In [ ]:
# Entrenamiento
#
def train_estimator(alpha=0.5, l1_ratio=0.5, verbose=1):
    """
    # Versión modularizada de la función de entrenamiento del modelo ElasticNet
    #
    # Parámetros:
    #   alpha: Parámetro de regularización (default=0.5)
    #   l1_ratio: Proporción de penalización L1 frente a L2 (default=0.5)
    #   verbose: Si es > 0, imprime métricas de evaluación (default=1)
    #
    # Esta función:
    # 1. Utiliza funciones auxiliares modularizadas para cada paso del proceso
    # 2. Carga los datos, realiza la división train/test
    # 3. Entrena el modelo ElasticNet con los parámetros proporcionados
    # 4. Evalúa el modelo y opcionalmente reporta las métricas
    # 5. Guarda el modelo si es mejor que el modelo previamente guardado
    #
    # Esta implementación es más modular y mantenible que la original
    """

    from sklearn.linear_model import ElasticNet

    x, y = load_data()
    x_train, x_test, y_train, y_test = make_train_test_split(x, y)
    estimator = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=12345)
    estimator.fit(x_train, y_train)
    mse, mae, r2 = eval_metrics(y_test, y_pred=estimator.predict(x_test))
    if verbose > 0:
        report(estimator, mse, mae, r2)

    best_estimator = load_best_estimator()
    if best_estimator is None or estimator.score(x_test, y_test) > best_estimator.score(
        x_test, y_test
    ):
        best_estimator = estimator

    save_best_estimator(best_estimator)


In [ ]:
###########
# Experimentación con tres configuraciones diferentes de hiperparámetros
# 
# Probamos tres combinaciones distintas de alpha y l1_ratio:
# 1. alpha=0.5, l1_ratio=0.5: Regularización media con balance entre L1 y L2
# 2. alpha=0.2, l1_ratio=0.2: Regularización más suave con mayor peso de L2
# 3. alpha=0.1, l1_ratio=0.1: Regularización muy suave con mayor peso de L2
#
# El objetivo es comparar el rendimiento de estas configuraciones
# y guardar automáticamente el modelo con mejor desempeño
train_estimator(0.5, 0.5)
train_estimator(0.2, 0.2)
train_estimator(0.1, 0.1)





ElasticNet(alpha=0.5, random_state=12345):
  MSE: 0.5294843132862007
  MAE: 0.5894666734018875
  R2: 0.13368827268570616
ElasticNet(alpha=0.2, l1_ratio=0.2, random_state=12345):
  MSE: 0.43869119518947153
  MAE: 0.5236106762028768
  R2: 0.2822387414965034
ElasticNet(alpha=0.1, l1_ratio=0.1, random_state=12345):
  MSE: 0.4183271587407731
  MAE: 0.5055024368693067
  R2: 0.31555720466583137


In [ ]:
def check_estimator():
    """
    # Función para verificar el rendimiento del mejor modelo guardado
    #
    # Esta función:
    # 1. Carga los datos y los divide en conjuntos de entrenamiento y prueba
    # 2. Carga el mejor modelo guardado (el que obtuvo mejor puntuación)
    # 3. Evalúa el modelo con los datos de prueba
    # 4. Imprime un reporte con las métricas de rendimiento
    #
    # Sirve para confirmar cuál es el modelo actual guardado y su rendimiento
    """

    x, y = load_data()
    x_train, x_test, y_train, y_test = make_train_test_split(x, y)
    estimator = load_best_estimator()
    mse, mae, r2 = eval_metrics(y_test, y_pred=estimator.predict(x_test))
    report(estimator, mse, mae, r2)


#
# Debe coincidir con el mejor modelo encontrado en la celdas anteriores
#
check_estimator()


ElasticNet(alpha=np.float64(0.0001), l1_ratio=np.float64(0.0001),
           random_state=12345):
  MSE: 0.40021745821413146
  MAE: 0.4848004855172136
  R2: 0.34518725328239785


In [ ]:
def make_hyperparameters_search(alphas, l1_ratios):
    """
    # Función para realizar una búsqueda exhaustiva de hiperparámetros
    #
    # Parámetros:
    #   alphas: Lista o array de valores para el parámetro alpha a probar
    #   l1_ratios: Lista o array de valores para el parámetro l1_ratio a probar
    #
    # Esta función:
    # 1. Itera sobre todas las combinaciones posibles de alpha y l1_ratio
    # 2. Entrena un modelo para cada combinación
    # 3. No muestra las métricas de cada modelo (verbose=0)
    # 4. Guarda automáticamente el mejor modelo encontrado
    #
    # La búsqueda de hiperparámetros es una técnica para encontrar la mejor 
    # configuración de parámetros para maximizar el rendimiento del modelo
    """

    for alpha in alphas:
        for l1_ratio in l1_ratios:
            train_estimator(alpha=alpha, l1_ratio=l1_ratio, verbose=0)



In [ ]:
import numpy as np

# Creación de arrays con valores de hiperparámetros a probar
# - alphas: 10 valores equidistantes entre 0.0001 y 0.5 para el parámetro de regularización
# - l1_ratios: 10 valores equidistantes entre 0.0001 y 0.5 para la proporción L1/L2
alphas = np.linspace(0.0001, 0.5, 10)
l1_ratios = np.linspace(0.0001, 0.5, 10)

# Ejecución de la búsqueda exhaustiva de hiperparámetros
# Esto entrenará 10x10=100 modelos diferentes con todas las combinaciones posibles
make_hyperparameters_search(alphas, l1_ratios)

# Verificación del mejor modelo encontrado durante la búsqueda de hiperparámetros
# El modelo guardado debe ser el que obtuvo el mejor rendimiento entre todas las combinaciones
check_estimator()


ElasticNet(alpha=np.float64(0.0001), l1_ratio=np.float64(0.0001),
           random_state=12345):
  MSE: 0.40021745821413146
  MAE: 0.4848004855172136
  R2: 0.34518725328239785


In [ ]:
def train_estimator(alphas, l1_ratios, n_splits=5, verbose=1):
    """
    # Versión avanzada de entrenamiento usando GridSearchCV para búsqueda automática de hiperparámetros
    #
    # Parámetros:
    #   alphas: Lista o array de valores para alpha a probar
    #   l1_ratios: Lista o array de valores para l1_ratio a probar
    #   n_splits: Número de divisiones para validación cruzada (default=5)
    #   verbose: Si es > 0, imprime métricas de evaluación (default=1)
    #
    # Esta función:
    # 1. Utiliza GridSearchCV que prueba todas las combinaciones posibles de hiperparámetros
    # 2. Realiza validación cruzada para evaluar modelos de forma más robusta
    # 3. Selecciona automáticamente el mejor modelo entre todas las combinaciones
    # 4. Guarda el mejor modelo si supera al modelo anterior
    #
    # GridSearchCV es más eficiente que la búsqueda manual y provee validación cruzada
    """

    from sklearn.linear_model import ElasticNet
    from sklearn.model_selection import GridSearchCV

    x, y = load_data()
    x_train, x_test, y_train, y_test = make_train_test_split(x, y)

    # -------------------------------------------------------------------------
    # Búsqueda de parámetros con validación cruzada
    #
    estimator = GridSearchCV(
        estimator=ElasticNet(
            random_state=12345,
        ),
        param_grid={
            "alpha": alphas,  # Todos los valores de alpha a probar
            "l1_ratio": l1_ratios,  # Todos los valores de l1_ratio a probar
        },
        cv=n_splits,  # Número de divisiones para validación cruzada
        refit=True,   # Reentrenar con los mejores parámetros
        verbose=0,    # No mostrar progreso detallado
        return_train_score=False,  # No devolver puntuaciones de entrenamiento
    )
    # -------------------------------------------------------------------------

    # Entrenamiento del GridSearchCV que prueba todas las combinaciones
    estimator.fit(x_train, y_train)

    # Extracción del mejor estimador encontrado
    estimator = estimator.best_estimator_

    # Evaluación del mejor modelo en el conjunto de prueba
    mse, mae, r2 = eval_metrics(y_test, y_pred=estimator.predict(x_test))
    if verbose > 0:
        report(estimator, mse, mae, r2)

    # Guarda el modelo si es mejor que el anterior
    best_estimator = load_best_estimator()
    if best_estimator is None or estimator.score(x_test, y_test) > best_estimator.score(
        x_test, y_test
    ):
        best_estimator = estimator

    save_best_estimator(best_estimator)



In [ ]:
###########
# Ejecución del entrenamiento con GridSearchCV
import numpy as np

# Ejecutamos la búsqueda de hiperparámetros con GridSearchCV que es más eficiente y robusta
# 
# Parámetros:
# - alphas: 10 valores espaciados entre 0.0001 y 0.5 
# - l1_ratios: 10 valores espaciados entre 0.0001 y 0.5
# - n_splits=5: Validación cruzada con 5 divisiones para evaluar cada combinación
# - verbose=1: Mostrar resultados del mejor modelo encontrado
#
# GridSearchCV probará 10x10=100 combinaciones de hiperparámetros
# pero con validación cruzada (5-fold), lo que resulta en 500 modelos entrenados
train_estimator(
    alphas=np.linspace(0.0001, 0.5, 10),
    l1_ratios=np.linspace(0.0001, 0.5, 10),
    n_splits=5,
    verbose=1,
)


ElasticNet(alpha=np.float64(0.0001), l1_ratio=np.float64(0.0001),
           random_state=12345):
  MSE: 0.40021745821413146
  MAE: 0.4848004855172136
  R2: 0.34518725328239785


In [ ]:
# Verificación final del mejor modelo encontrado
#
# Después de todas las búsquedas de hiperparámetros realizadas:
# 1. Búsqueda manual con tres combinaciones específicas
# 2. Búsqueda exhaustiva con bucles anidados (100 combinaciones)
# 3. Búsqueda con GridSearchCV y validación cruzada (500 evaluaciones)
#
# Esta llamada nos muestra las métricas finales del mejor modelo guardado
# y su configuración de hiperparámetros (valores de alpha y l1_ratio)
check_estimator()

ElasticNet(alpha=np.float64(0.0001), l1_ratio=np.float64(0.0001),
           random_state=12345):
  MSE: 0.40021745821413146
  MAE: 0.4848004855172136
  R2: 0.34518725328239785
